In [1]:
from lxml import etree

In [2]:
college = 'Algonquin'

infile = 'input/' + college +'_invoices_20211123.xml'
outfile = 'output/' + college +'_invoices_fixed_20211126.xml'

In [23]:
# Specify the XMLParser with recover option to catch any '&' that 
# have not been properly escaped in the source file.
tree = etree.parse(infile, parser=etree.XMLParser(recover=True))


In [24]:
# Start by setting all currency attributes to CAD
currencies = tree.xpath('//*[@currency]')
for currency in currencies:
    currency.attrib['currency']='CAD'

In [25]:
# Where exchangeRate is present, distinguish between native amount and vendor amount
converted_amounts = tree.xpath('//exchangeRate')
print(len(converted_amounts))

for amount in converted_amounts:
    if amount.text is not None:
        
        # Fix exchange rate
        forex = 1/float(amount.text)
        amount.text = format(forex, '.2f')
        
        # Specify currency and apply correct exchange rate to native (converted) amount and vendor amount:
        native_amounts = amount.xpath('../../..//nativeAmount|../../..//nativeAmountPaid|../../..//amountPaid')
        for native in native_amounts:
            native.attrib['currency']='CAD'
            native.text = format(float(native.text)/forex, '.2f')

        converted_amounts = amount.xpath('../../..//vendorAmount|../../..//vendorAmountPaid|../../..//vendorFinalPrice|../../..//remitToVendor')
        for converted in converted_amounts:
            converted.attrib['currency']='USD'
            converted.text = format(float(converted.text)*forex, '.2f')
    
tree.write(outfile, pretty_print=True)

61
